## Import packages

In [ ]:
!pip3 install -qq twint
!pip install -qq whatthelang
!pip install snscrape

     |████████████████████████████████| 1.3 MB 14.3 MB/s 
     |████████████████████████████████| 263 kB 74.1 MB/s 
     |████████████████████████████████| 364 kB 68.6 MB/s 
     |████████████████████████████████| 291 kB 60.8 MB/s 
     |████████████████████████████████| 142 kB 56.9 MB/s 
     |████████████████████████████████| 294 kB 89.0 MB/s 
     |████████████████████████████████| 786 kB 7.1 MB/s 
     |████████████████████████████████| 455 kB 56.7 MB/s 
     |████████████████████████████████| 244 kB 82.0 MB/s 


In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply()

## Tweet Scraping

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('socialism since:2020-01-01 until:2021-08-03').get_items()):
    if i>4000000:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])
    
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
tweets_df.head()

,Datetime,Tweet Id,Text,Username
0,2021-08-02 23:59:54+00:00,1422346450772631553,@whoistaneesh @gilb3rt101 @b3ckton ratio then,maxthesnowflak3
1,2021-08-02 23:59:44+00:00,1422346408934453252,@KelbPanthera @Timcast Interesting🤔 I’m having...,AliceOrtegaR
2,2021-08-02 23:59:39+00:00,1422346389053378567,@louie_jackjr @notcrypticno yeah and either wa...,maxthesnowflak3
3,2021-08-02 23:59:33+00:00,1422346361140400129,@NetBoyRick2pt0 @aaronjmate @PushbackShow Oh m...,howell_mg
4,2021-08-02 23:59:31+00:00,1422346353183690758,@Azure_Husky Tell them they're wonderful,ExpAllison


## Cleanup of Duplicates and Language

In [ ]:
# Cleanup
tweets = tweets_df.drop_duplicates(subset=['Username', 'Datetime', 'Text'])

In [ ]:
# Reindex
tweets.index = range(len(tweets))

In [ ]:
# Remove non-english
from whatthelang import WhatTheLang
wtl = WhatTheLang()

In [ ]:
# This function makes easy to handle exceptions (e.g. no text where text should be)
# not really needed but can be useful 

def detect_lang(text):
    try: 
        return wtl.predict_lang(text)
    except Exception:
        return 'exp'

In [ ]:
# Added performance measure here...you can leave teh %%time line out

%%time

tweets['lang'] = tweets['Text'].map(lambda t: detect_lang(t))

CPU times: user 30.5 s, sys: 115 ms, total: 30.7 s
Wall time: 30.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# keep only english

tweets = tweets[tweets.lang == 'en']

In [ ]:
print(f"all tweets: {tweets.shape}")

all tweets: (789497, 5)


## Export the updated data

In [ ]:
tweets.to_csv('socialism-tweets.csv', sep=',', index=False)